This tutorial demonstrates how to train a sequence-to-sequence (seq2seq) model for Spanish-to-English translation.

# Setup

In [ ]:
!pip install "tensorflow-text>=2.10"
!pip install einops

In [ ]:
import numpy as np

import typing
from typing import Any, Tuple

import einops
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tensorflow as tf
import tensorflow_text as tf_text

This tutorial uses a lot of low level API's where it's easy to get shapes wrong. This class is used to check shapes throughout the tutorial.

In [ ]:
class ShapeChecker():
  def __init__(self):
    # Keep a cache of every axis-name seen
    self.shapes = {}

  def __call__(self, tensor, names, broadcast=False):
    if not tf.executing_eagerly():
      return

    parsed = einops.parse_shape(tensor, names)

    for name, new_dim in parsed.items():
      old_dim = self.shapes.get(name, None)
      
      if (broadcast and new_dim == 1):
        continue

      if old_dim is None:
        # If the axis name is new, add its length to the cache.
        self.shapes[name] = new_dim
        continue

      if new_dim != old_dim:
        raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                         f"    found: {new_dim}\n"
                         f"    expected: {old_dim}\n")

# Data

The tutorial uses a language dataset provided by Anki (http://www.manythings.org/anki/)

## Donwload and prepare the dataset

For convenience, a copy of this dataset is hosted on Google Cloud, but you can also download your own copy. After downloading the dataset, here are the steps you need to take to prepare the data:


1.   Add a *start* and *end* token to each sentence.
2.   Clean the sentences by removing special characters.
3.   Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
4.   Pad each sentence to a maximun length.



In [ ]:
# Download the file
import pathlib

path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = pathlib.Path(path_to_zip).parent/'spa-eng/spa.txt'

In [ ]:
def load_data(path):
  text = path.read_text(encoding = 'utf-8')

  lines = text.splitlines()
  pairs = [line.split('\t') for line in lines]

  context = np.array([context for target, context in pairs])
  target = np.array([target for target, context in pairs])

  return target, context

In [ ]:
target_raw, context_raw = load_data(path_to_file)
print(target_raw[-1])
print(context_raw[-1])

In [ ]:
print("Number of examples:", len(context_raw))

## Create a dataset

From these arrays of string you can create a *tf.data.Dataset* of string that shuffles and batches them efficiently

In [ ]:
BUFFER_SIZE = len(context_raw)
BATCH_SIZE = 64

In [ ]:
is_train = np.random.uniform(size = (len(target_raw),)) < 0.8

In [ ]:
np.unique(is_train, return_counts=True)
# 23810 examples for val
# 95154 examples for train

In [ ]:
train_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[is_train], target_raw[is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
)

In [ ]:
val_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[~is_train], target_raw[~is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
)

In [ ]:
for example_context_strings, example_target_strings in train_raw.take(1):
  print(example_context_strings[:5])
  print()
  print(example_target_strings[:5])

## Text preprocessing

One of the goals of this tutorial is to build a model that can be exported as a *tf.saved_model*. To make that exported model useful it should take *tf.string* inputs, and return *tf.string* outputs: All the text processing happens inside the model. Mainly using a layers.TextVectorization layer.

### Standardization

The model is dealing with multilingual text with a limited vocabulary. So it will be important to standardize the input text.

The first step is Unicode normalization to split accented characters and replace compatibility characters with their ASCII equivalents.

In [ ]:
example_text = tf.constant('¿Todavía está en casa?')

print(example_text.numpy())
print(tf_text.normalize_utf8(example_text, 'NFKD').numpy())

Unicode normalization will be the first step in the text standardization function

In [ ]:
def tf_lower_and_split_punct(text):
  # Split accented characters
  text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)

  # Keep space, a to z, and select punctuation
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')

  # Add spaces around punctuation
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')

  # Strip whitespace
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator = ' ')
  return text

In [ ]:
print(example_text.numpy().decode()) # before noirmalization
print(tf_lower_and_split_punct(example_text).numpy().decode()) # after normalization

### Text Vectorization

This standardization function will be wrapped up in a *tf.keras.layers.TextVectorization* layer which will handle the vocabulary extraction and conversion of input text to sequences of tokens.

In [ ]:
max_vocab_size = 5000

context_text_processor = tf.keras.layers.TextVectorization(
    standardize = tf_lower_and_split_punct,
    max_tokens = max_vocab_size,
    ragged = True,
)

The *TextVectorization* layer and many other Keras preprocessing layers have an adapt method. This method reads one epoch of the training data, and works a lot like *Model.fit*. This adapt method initializes the layer based on the data. Here it determines the vocabulary:

In [ ]:
context_text_processor.adapt(train_raw.map(lambda context, target: context))

# Here are the first 10 words from the vocabulary:
context_text_processor.get_vocabulary()[:10]

That's the Spanish TextVectorization layer, now build and .adapt() the English one:

In [ ]:
target_text_processor = tf.keras.layers.TextVectorization(
    standardize = tf_lower_and_split_punct,
    max_tokens = max_vocab_size,
    ragged = True,
)

target_text_processor.adapt(train_raw.map(lambda context, target: target))
target_text_processor.get_vocabulary()[:10]

Now these layers can convert a batch of strings into a batch of token IDs:

In [ ]:
example_tokens = context_text_processor(example_context_strings)
example_tokens[:3, :]

The *get_vocabulary* method can be used to convert token IDs back to text:

In [ ]:
context_vocab = np.array(context_text_processor.get_vocabulary())
tokens = context_vocab[example_tokens[0].numpy()]

' '.join(tokens)

The returned token IDs are zero-padded. This can easily be turned into a mask:

In [ ]:
plt.subplot(1, 2, 1)
plt.pcolormesh(example_tokens.to_tensor())
plt.title('Token IDs')

plt.subplot(1, 2, 2)
plt.pcolormesh(example_tokens.to_tensor() != 0)
plt.title('Mask')

## Process the dataset

The process_text function below converts the Datasets of strings, into 0-padded tensors of token IDs. It also converts from a (context, target) pair to an ((context, target_in), target_out) pair for training with keras.Model.fit. Keras expects (inputs, labels) pairs, the inputs are the (context, target_in) and the labels are target_out. The difference between target_in and target_out is that they are shifted by one step relative to eachother, so that at each location the label is the next token.



In [ ]:
def process_text(context, target):
  context = context_text_processor(context).to_tensor()
  target = target_text_processor(target)

  targ_in = target[:,:-1].to_tensor()
  targ_out = target[:,1:].to_tensor()
  
  return (context, targ_in), targ_out


train_ds = train_raw.map(process_text, tf.data.AUTOTUNE)
val_ds = val_raw.map(process_text, tf.data.AUTOTUNE)

In [ ]:
for (ex_context_tok, ex_tar_in), ex_tar_out in train_ds.take(1):
  print(ex_context_tok[0, :10].numpy()) 
  print()
  print(ex_tar_in[0, :10].numpy()) 
  print(ex_tar_out[0, :10].numpy()) 